In [1]:
### Importing packages
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import os
import sqlite3
import plotly.express as px
import us
from ipywidgets import widgets, interact
import plotly.graph_objects as go

In [2]:
### Load the data
df = pd.read_excel("Health_Science_Dataset.xlsx", header=1)

In [3]:
### Header & Tail for Data
df

,Data As Of,Start Week,End Week,MMWRyear,MMWRweek,Week Ending Date,Group,Indicator,Jurisdiction,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Influenza Deaths,Pneumonia or Influenza,"Pneumonia, Influenza, or COVID-19 Deaths"
0,11/02/2023,12/29/2019,01/04/2020,2020,1,01/04/2020,By Week,Week-ending,United States,All Ages,0.0,60028.0,4102.0,432.0,4534.0,4534.0
1,11/02/2023,12/29/2019,01/04/2020,2020,1,01/04/2020,By Week,Week-ending,United States,0-17 years,0.0,667.0,19.0,22.0,41.0,41.0
2,11/02/2023,12/29/2019,01/04/2020,2020,1,01/04/2020,By Week,Week-ending,United States,18-64 years,0.0,14706.0,767.0,183.0,950.0,950.0
3,11/02/2023,12/29/2019,01/04/2020,2020,1,01/04/2020,By Week,Week-ending,United States,65 years and over,0.0,44655.0,3316.0,227.0,3543.0,3543.0
4,11/02/2023,12/29/2019,01/04/2020,2020,1,01/04/2020,By Week,Week-ending,Alabama,All Ages,0.0,1098.0,67.0,NaN,72.0,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50395,11/02/2023,10/22/2023,10/28/2023,2023,43,10/28/2023,By Week,Week-ending,HHS Region 9,65 years and over,43.0,1970.0,128.0,NaN,129.0,155.0
50396,11/02/2023,10/22/2023,10/28/2023,2023,43,10/28/2023,By Week,Week-ending,HHS Region 10,All Ages,14.0,435.0,26.0,0.0,26.0,36.0
50397,11/02/2023,10/22/2023,10/28/2023,2023,43,10/28/2023,By Week,Week-ending,HHS Region 10,0-17 years,0.0,0.0,0.0,0.0,0.0,0.0
50398,11/02/2023,10/22/2023,10/28/2023,2023,43,10/28/2023,By Week,Week-ending,HHS Region 10,18-64 years,0.0,61.0,NaN,0.0,NaN,NaN


In [4]:
### Column names before cleaning
[f"{col}: {df[col].dtype}" for col in df.columns]

['Data As Of: object',
 'Start Week: object',
 'End Week: object',
 'MMWRyear: int64',
 'MMWRweek: int64',
 'Week Ending Date: object',
 'Group: object',
 'Indicator: object',
 'Jurisdiction: object',
 'Age Group: object',
 'COVID-19 Deaths: float64',
 'Total Deaths: float64',
 'Pneumonia Deaths: float64',
 'Influenza Deaths: float64',
 'Pneumonia or Influenza: float64',
 'Pneumonia, Influenza, or COVID-19 Deaths: float64']

In [5]:
# String
for col_idx in [6, 7, 8, 9]:
    col_name = df.columns[col_idx]
    df[col_name] = df[col_name].astype(str)

# Dates
for col_idx in [0, 1, 2, 5]:
    col_name = df.columns[col_idx]
    df[col_name] = pd.to_datetime(df[col_name], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Year
df['MMWRyear'] = df['MMWRyear'].astype(int)

# Integer
for col_idx in [4] + list(range(10, 16)):
    col_name = df.columns[col_idx]
    df[col_name] = df[col_name].apply(lambda x: int(x) if pd.notna(x) else x)

In [6]:
### Column names after cleaning
[f"{col}: {df[col].dtype}" for col in df.columns]

['Data As Of: datetime64[ns]',
 'Start Week: datetime64[ns]',
 'End Week: datetime64[ns]',
 'MMWRyear: int64',
 'MMWRweek: int64',
 'Week Ending Date: datetime64[ns]',
 'Group: object',
 'Indicator: object',
 'Jurisdiction: object',
 'Age Group: object',
 'COVID-19 Deaths: float64',
 'Total Deaths: float64',
 'Pneumonia Deaths: float64',
 'Influenza Deaths: float64',
 'Pneumonia or Influenza: float64',
 'Pneumonia, Influenza, or COVID-19 Deaths: float64']

In [7]:
# Define a list of death metrics
death_metrics = [
    'Total Deaths',
    'Pneumonia Deaths',
    'Influenza Deaths',
    'Pneumonia or Influenza',
    'Pneumonia, Influenza, or COVID-19 Deaths'
]

# Map 'Jurisdiction' to state abbreviations
state_names = [state.name for state in us.states.STATES]
state_abbrevs = [state.abbr for state in us.states.STATES]
name_to_abbr = {state.name: state.abbr for state in us.states.STATES}

def map_jurisdiction(jurisdiction):
    if jurisdiction in state_abbrevs:
        return jurisdiction
    elif jurisdiction in name_to_abbr:
        return name_to_abbr[jurisdiction]
    else:
        return None  # For unrecognized jurisdictions

df['State_Abbr'] = df['Jurisdiction'].apply(map_jurisdiction)

# Filter out unrecognized states
df = df[df['State_Abbr'].notna()].copy()

# Map 'State_Abbr' to FIPS codes
abbr_to_fips = {state.abbr: state.fips for state in us.states.STATES}
df['FIPS'] = df['State_Abbr'].map(abbr_to_fips)

In [8]:
# Function to create the choropleth map
def create_choropleth_with_year_annotation(death_metric):
    # Ensure 'Date' and 'Date_Str' columns exist
    if 'Date' not in df.columns:
        # Construct 'Year-Week' string
        df['Year-Week'] = df['MMWRyear'].astype(str) + '-W' + df['MMWRweek'].astype(str).str.zfill(2)
        # Convert 'Year-Week' to datetime using ISO week date format
        df['Date'] = pd.to_datetime(df['Year-Week'] + '-1', format='%G-W%V-%u', errors='coerce')
    
    if 'Date_Str' not in df.columns:
        df['Date_Str'] = df['Date'].dt.strftime('%Y-%m-%d')

    # Get unique dates and sort them
    dates = sorted(df['Date'].unique())

    # Create initial figure
    date = dates[0]
    df_date = df[df['Date'] == date]

    fig = go.Figure()

    # Add initial data
    fig.add_trace(go.Choropleth(
        locations=df_date['State_Abbr'],
        z=df_date[death_metric],
        locationmode='USA-states',
        colorscale='Reds',
        colorbar_title='Number of Deaths',
        hovertext=df_date['Jurisdiction'],
        hoverinfo='text+z',
    ))

    # Add initial annotation for the year
    initial_year = pd.to_datetime(date).year
    fig.update_layout(
        annotations=[
            dict(
                x=0.5,
                y=1.05,
                xref='paper',
                yref='paper',
                text=f"Year: {initial_year}",
                showarrow=False,
                font=dict(size=16),
                align='center',
            )
        ]
    )

    # Create frames
    frames = []
    for date in dates:
        df_date = df[df['Date'] == date]
        year_str = pd.to_datetime(date).year
        frame = go.Frame(
            data=[go.Choropleth(
                locations=df_date['State_Abbr'],
                z=df_date[death_metric],
                locationmode='USA-states',
                colorscale='Reds',
                hovertext=df_date['Jurisdiction'],
                hoverinfo='text+z',
            )],
            name=str(date),
            layout=go.Layout(
                annotations=[
                    dict(
                        x=0.5,
                        y=1.05,
                        xref='paper',
                        yref='paper',
                        text=f"Year: {year_str}",
                        showarrow=False,
                        font=dict(size=16),
                        align='center',
                    )
                ]
            )
        )
        frames.append(frame)

    fig.frames = frames

    # Create slider steps
    steps = []
    for date in dates:
        date_str = str(date)
        step = dict(
            method='animate',
            args=[
                [date_str],
                {
                    'mode': 'immediate',
                    'frame': {'duration': 500, 'redraw': True},
                    'transition': {'duration': 0}
                }
            ],
            label=''  # Remove labels to avoid clutter
        )
        steps.append(step)

    # Configure sliders
    sliders = [dict(
        active=0,
        currentvalue={
            'prefix': '',
            'font': {'size': 16, 'color': '#666'},
            'visible': False,
            'xanchor': 'right'
        },
        pad={'t': 50},
        steps=steps
    )]

    # Update layout with sliders and play button
    fig.update_layout(
        sliders=sliders,
        title_text=f'{death_metric} Over Time',
        title_x=0.5,
        geo_scope='usa',
        geo_projection_type='albers usa',
        updatemenus=[dict(
            type='buttons',
            showactive=False,
            buttons=[dict(
                label='Play',
                method='animate',
                args=[None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}]
            )],
            x=0.1,
            y=0,
            xanchor='right',
            yanchor='top'
        )]
    )

    fig.show()


# Function to create the interactive choropleth map
def interactive_choropleth():
    death_metric_widget = widgets.Dropdown(
        options=death_metrics,
        value='Total Deaths',
        description='Death Metric:',
    )

    interact(create_choropleth_with_year_annotation, death_metric=death_metric_widget)

In [9]:
interactive_choropleth()

interactive(children=(Dropdown(description='Death Metric:', options=('Total Deaths', 'Pneumonia Deaths', 'Infl…